In [1]:
import torch
from kernel_utils import VideoReader, FaceExtractor, confident_strategy, predict_on_video_set
from training.zoo.classifiers import DeepFakeClassifier

import urllib.request
import os
import moviepy.editor as mp
import re
import logging
from config import create_api
import tweepy
import time

In [2]:
weights_folder = './weights'
models = []
model_paths = [os.path.join(weights_folder, model) for model in os.listdir(weights_folder)]
for path in model_paths:
    model = DeepFakeClassifier(encoder="tf_efficientnet_b7_ns").to("cuda")
    print("loading state dict {}".format(path))
    checkpoint = torch.load(path, map_location="cpu")
    state_dict = checkpoint.get("state_dict", checkpoint)
    model.load_state_dict({re.sub("^module.", "", k): v for k, v in state_dict.items()}, strict=True)
    model.eval()
    del checkpoint
    models.append(model.half())

loading state dict ./weights\final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36


In [3]:
def detect_deepfake_module(tweet):
    video_link_list = tweet.extended_entities['media'][0]['video_info']['variants']

    top_res = []
    for vll in video_link_list:
        if 'bitrate' in vll:
            top_res.append([vll['bitrate'], vll['url']])

    best_vid_link = sorted(top_res, reverse=True)[0][1]
    
    print(best_vid_link)
    
    urllib.request.urlretrieve(best_vid_link, './tweet_videos/tweet_deepfake_detect.mp4') 
    
    clip = mp.VideoFileClip("./tweet_videos/tweet_deepfake_detect.mp4")
    clip_resized = clip.resize(height=1080) # make the height 360px ( According to moviePy documenation The width is then computed so that the width/height ratio is conserved.)
    clip_resized.write_videofile("./tweet_videos/tweet_deepfake_detect_1080p.mp4")
    
    os.remove('./tweet_videos/tweet_deepfake_detect.mp4')
    
    frames_per_video = 32
    video_reader = VideoReader()
    video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
    face_extractor = FaceExtractor(video_read_fn)
    input_size = 380
    strategy = confident_strategy
    stime = time.time()

    video_folder = './tweet_videos'
    test_videos = os.listdir(video_folder)
    print("Predicting {} videos".format(len(test_videos)))
    predictions = predict_on_video_set(face_extractor=face_extractor, input_size=input_size, models=models,
                                       strategy=strategy, frames_per_video=frames_per_video, videos=test_videos,
                                       num_workers=6, test_dir=video_folder)
    
    
    os.remove('./tweet_videos/tweet_deepfake_detect_1080p.mp4')
    
    return "AI response: Fake" if round(predictions[0]) else "AI response: Not a Fake"

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

def check_mentions(api, since_id):
    logger.info("Retrieving mentions")
    new_since_id = since_id
    for tweet in tweepy.Cursor(api.mentions_timeline,
        since_id=since_id).items():
        new_since_id = max(tweet.id, new_since_id)
        
        if tweet.in_reply_to_status_id is not None:
            continue
            
        if 'extended_entities' not in tweet._json:
            continue
            
        if any(keyword in tweet.text.lower() for keyword in ['detect', 'ai']):
            logger.info(f"Using AI for {tweet.user.name}, to detect deepfake")
            
            print(tweet.text.lower())

            output = detect_deepfake_module(tweet)
            
            print("Also sending to Toloka for analysis and feedback!")
            
            api.update_status(
                status=output,
                in_reply_to_status_id=tweet.id,
            )

        elif any(keyword in tweet.text.lower() for keyword in ['detect', 'human']):
            logger.info(f"Using Human experts for {tweet.user.name}, to detect deepfake")
            
            print(tweet.text.lower())

            api.update_status(
                status="Forwarded to digital forensics team, Will generate report and respond shortly!",
                in_reply_to_status_id=tweet.id,
            )
            
    return new_since_id

def main():
    api = create_api()
    # doing this to avoid collecting older tweets and responding to it
    for tweet in tweepy.Cursor(api.mentions_timeline,since_id=1).items():
        print(tweet.id)
        break
    since_id = tweet.id
#     since_id = 1
    while True:
        since_id = check_mentions(api, since_id)
        logger.info("Waiting...")
        time.sleep(30)

In [5]:
main()

INFO:root:API created
INFO:root:Retrieving mentions


1469119815843397633


INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Using AI for Aswin Shriram (academic), to detect deepfake


@aswinacademic detect deepfake in this video using ai https://t.co/qroiwobpz0
https://video.twimg.com/ext_tw_video/1469137162390016000/pu/vid/720x1280/rxb98HDJ6hZd4v5i.mp4?tag=12
Moviepy - Building video ./tweet_videos/tweet_deepfake_detect_1080p.mp4.
MoviePy - Writing audio in tweet_deepfake_detect_1080pTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./tweet_videos/tweet_deepfake_detect_1080p.mp4



t: 100%|███████████████████████████████████████████████████████████████████| 299/299 [00:06<00:00, 48.20it/s, now=None]WARNING:py.warnings:C:\Users\aswin\AppData\Roaming\Python\Python37\site-packages\moviepy\video\io\ffmpeg_reader.py:130: UserWarning: Warning: in file ./tweet_videos/tweet_deepfake_detect.mp4, 2764800 bytes wanted but 0 bytes read,at frame 298/299, at time 10.02/10.03 sec. Using the last valid frame instead.
  UserWarning)



Moviepy - Done !
Moviepy - video ready ./tweet_videos/tweet_deepfake_detect_1080p.mp4
Predicting 1 videos
Also sending to Toloka for analysis and feedback!


INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...


KeyboardInterrupt: 